## short-circuit

short-circuit happens when the operator reaches an operand that allows them to make an conclusion about the expression. For example, `and` will short-circuit as soon as it reaches the first false value because it then knows that not all the values are true.

In [2]:
1/0

ZeroDivisionError: division by zero

In [1]:
True or 1/0

True

In [3]:
True and 13

13

In [8]:
10 % 10

0

In [9]:
10 / 10

1.0

In [10]:
yes_no = input('i know the number')

i know the number2


In [12]:
yes_no.strip()

'2'

## Testing

In [1]:
def fib(n):
    '''
    计算 fibonacci 数 n>=2
    '''
    pred, curr = 0, 1 #fib 数列第一个数和第二个数
    k = 2
    while k < n:
        pred, curr = curr, pred + curr
        k = k + 1
    return curr

**Assertions**:  
使用 `assert` statements 来对比期望值，比如测试的函数的输出

In [9]:
assert fib(8) == 13, 'The 8th Fibonacci number should be 13'

**Doctests**  
python 提供了一个很便利的方法——可以在函数的 docstring 中放置简单的测试。docstring 第一行应当包含这个函数的描述，然后紧跟着是一个空行，接下来可能有详细的参数和函数行为的描述，除此之外还可能包含一些调用函数的样例

In [18]:
def sum_naturals(n):
    '''return sum of the first n natural numbers.
    
    >>> sum_naturals(10)
    55
    >>> sum_naturals(100)
    5050
    '''
    total, k = 0, 1
    while k <= n:
        total += k
        k += 1
    return total

我们可以使用 `doctest` 模块来做测试，`globals` 函数返回全局环境的一个表示，解释器需要它来评估 expressions. `run_docstring_examples` 函数，它的第一个参数是需要测试的函数，第二个参数是 expression `globals()` 返回的结果，第三个参数是 True 来表示我们想要 verbose 输出：就是一个所有测试运行的记录

In [19]:
from doctest import run_docstring_examples
run_docstring_examples(sum_naturals, globals(), True)

Finding tests in NoName
Trying:
    sum_naturals(10)
Expecting:
    55
ok
Trying:
    sum_naturals(100)
Expecting:
    5050
ok


## A Guide to designing Function

* 给每个函数一个准确的工作
* 不要重复自己，实现一个过程只一次但是运行它很多次
* 定义一个通用函数

## Higher-Order Functions

### **Functions as Arguments**

In [21]:
def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total

def cube(x):
    return x**3

def pi_term(x):
    return 8 / ((4*x-3) * (4*x-1))

In [24]:
summation(20000, pi_term)

3.1415676535897927

### **Functions as General Methods**

考虑下面为了迭代更新而实现一个 general method，并用它来计算黄金分割比。一个迭代更新算法初始化一个 `guess` 作为等式的解，它不断的应用 `update` 函数来更新 `guess`，并应用 `close` 的比较来判别是否当前的 `guess` 足够接近正确的值。

In [25]:
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess

这个函数不表明解决什么样的问题：细节被丢给了作为 arguments 传入的 `update` 和 `close` 函数

我们下面定义的迭代黄金分割比的 `golden_update` 和 `square_close_to_successor` 函数

In [26]:
def golden_update(guess):
    return 1/guess + 1

def square_close_to_successor(guess):
    return approx_eq(guess*guess, guess+1)

def approx_eq(x, y, tolerance=1e-15):
    return abs(x - y) < tolerance

In [27]:
improve(golden_update, square_close_to_successor)

1.6180339887498951

由此可见，命名和函数让我们抽象了巨大的复杂性，每个函数的定义可能是很简单琐碎的，但是整个计算过程确是很复杂，我们通过将小的部件组合形成复杂的过程

### **Nested Definitions** 

上面我们介绍了可以在函数中传入函数，这样增加了我们的编程语言的表现力，然而里面有一个问题是 `update` 之于 `improve` 函数只能有一个参数，**nested function** 可以帮我们解决这个问题，但是需要丰富我们的 environment model

让我们考虑一个新的问题：求一个数的平方根。下面我们想要求 a 的平方根，不断重复下面的更新会收敛到 \\(\sqrt a\\)

In [28]:
def average(x, y):
    return (x + y)/2

def sqrt_update(x, a):
    return average(x, a/x)

上面具有两个 argument 的更新函数与 `improve` 函数是不相符合的，解决方法是放置函数定义在其他定义中

In [29]:
def sqrt(a):
    def sqrt_update(x):
        return average(x, a/x)
    def sqrt_close(x):
        return approx_eq(x*x, a)
    return improve(sqrt_update, sqrt_close)

**Lexical scope(词法范围)**：nested functions 可以访问环境中它们定义时的 names（这里感觉翻译为变量比较好，比如这里的 a）

我们定义对 environment model 的两个拓展来了解 lexical scoping:
* 每个用户定义的函数都有一个 parent environment，函数都是在其中被定义的
* 当一个用户定义的函数被调用时，它的 local frame 就会拓展到 parent environment

在调用 `sqrt(256)` 时，environment 先为 `sqrt` 增加一个 local frame，然后评估 `sqrt_update` 和 `sqrt_close` 的 def statements 

![image](1.jpg)

注意当调用用户定义的函数时，产生的 local frame 和 这个函数具有相同的 parent

我们可以看到下面的 f5 frame 中，只包含了 x，但是它的 parent frame f1 仍然包含 a 

![image](2.jpg)

python 首先看 `sqrt_update` frame 中 —— 没有 a 存在，然后看它的 parent frame f1, 发现了 a 的 binding 值 256. `sqrt_update` 函数携带了一些数据: a 的值（当它被定义的时候），因为它 "enclose" 信息，所以 locally defined functions 通常被叫做 *closures(闭包)*

### Functions as returned values

词法范围编程语言的一个重要特征就是，在 local 定义的函数在返回时，保持了它们的 parent 环境

In [1]:
def compose1(f, g):
    def h(x):
        return f(g(x))
    return h

当我们定义了很多简单的函数之后，可以使用函数复合 (function composition)来组合这些函数，比如给定了 `f(x)` 和 `g(x)` 之后，我们想定义 `h(x) = f(g(x))`，如上面的代码

![image](3.jpg)

### Currying

我们可以用 high-order functions 来将一个有很多参数的函数变成一个每个函数只有一个参数的函数链。比如一个函数 `f(x,y)` ,我们可以定义一个新的函数 `g` 使得 `g(x)(y)` 与 `f(x,y)` 一样，这里 `g` 是一个 以 x 作为参数，并且返回的函数以 y 的参数的一个 high-order function，这个转变我们叫做 currying （我不知道怎么翻译成中文）

我们可以定义一个 curried 版本的 `pow` 函数 

In [2]:
def curried_pow(x):
    def h(y):
        return pow(x, y)
    return h

In [3]:
curried_pow(3)(4)

81

当我们只需要带有一个参数的函数时， currying 是非常有用的， the map pattern 就是对一组数应用单参数函数。在后面的篇章中我们会看到the map pattern 更多的例子，现在我们在一个函数中实现这个 pattern

In [4]:
def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start = start + 1

In [5]:
map_to_range(0, 10, curried_pow(2))

1
2
4
8
16
32
64
128
256
512


在上面的函数中，我们对 `pow` 函数实施了 currying transformation 来得到 `curried_pow`，现在我们可以定义函数自动 currying,也可以 uncurrying

In [6]:
def curry2(f):
    '''返回了一个两参数函数的 curried 版本'''
    def g(x):
        def h(y):
            return f(x, y)
        return h
    return g

In [7]:
def uncurry2(g):
    '''返回一个两参数版本的给定的 curried 函数'''
    def f(x, y):
        return g(x)(y)
    return f

In [9]:
pow_curried = curry2(pow)

In [10]:
pow_curried(2)(5)

32

In [12]:
map_to_range(0, 10, pow_curried(2))

1
2
4
8
16
32
64
128
256
512


`curry2` 以一个二参数函数 `f` 作为参数，返回一个单参数函数 `g`，因此 `curry2(f)(x)(y)` 等于 `f(x, y)`. `uncurry2` 函数重置了 currying transformation, 因此 `uncurry2(curry2(f))` 等于 `f`

In [13]:
uncurry2(pow_curried)(2, 5)

32

### Lambda Expressions

之前我们定义函数都会给它定义一个名字，在 python 中我们可以定义使用 `lambda` 表达式来定义匿名函数。一个 lambda 表达式评估了一个具有单一返回表达式当作它的 body,赋值和控制声明都是不允许的。

In [14]:
# 利用 lambda 重新定义 compose1 函数
def compose1(f, g):
    return lambda x: f(g(x))

我们可以这样理解 lambda 表达式的结构：  
lambda          x          :           f(g(x))  
a function that takes x   and returns  f(g(x))

lambda 表达式的结果被叫做 lambda 函数，它没有名字（python 打印 <lambda> 作为名字）,但是它与其他函数没什么区别

In [15]:
s  = lambda x: x * x

In [16]:
s

<function __main__.<lambda>(x)>

In [17]:
s(12)

144

在 environment diagram 中，lambda 表达式的结果也是函数，它的名字是希腊字母 \\(\lambda \\)。我们的 compose 函数可以用 lambda 表达式非常简洁的来表述

![image](4.jpg)

但是使用匿名函数会使得程序的可读性变差，但是有的时候使用它会使得程序变得更简洁，比如前面举的例子，当你想返回函数的时候。

### Function Decorators

python 提供了一种特殊的语法来实现 higher-order 函数，作为执行 def 语句的一部分（被称作装饰器）

In [18]:
def trace(fn):
    def wrapped(x):
        print('->', fn, '(', x, ')')
        return fn(x)
    return wrapped

In [19]:
@trace
def triple(x):
    return 3 * x

In [20]:
triple(12)

-> <function triple at 0x10d791048> ( 12 )


36

注意在上面的 `triple` 函数的 def 前有一个  @ 标记，它影响了 def 的执行规则。如果是正常的状况，函数 triple 被创造了，但是现在这里面的名字 triple 不再绑定到这个函数。它绑定的是在 triple 上 调用 trace 的返回值，这个装饰器与下面的定义相同：

In [21]:
def triple(x):
    return 3 * x
triple = trace(triple)